<a href="https://colab.research.google.com/github/Advaitt19/Hello-world/blob/main/XGBoost_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
import xgboost as xgb
from xgboost import XGBClassifier
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import xgboost as xgb

In [2]:
data = pd.read_csv("star_classification.csv")

In [3]:
data.head()

,obj_ID,alpha,delta,u,g,r,i,z,run_ID,rerun_ID,cam_col,field_ID,spec_obj_ID,class,redshift,plate,MJD,fiber_ID
0,1.237661e+18,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,3606,301,2,79,6.543777e+18,GALAXY,0.634794,5812,56354,171
1,1.237665e+18,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,4518,301,5,119,1.176014e+19,GALAXY,0.779136,10445,58158,427
2,1.237661e+18,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,3606,301,2,120,5.152200e+18,GALAXY,0.644195,4576,55592,299
3,1.237663e+18,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,4192,301,3,214,1.030107e+19,GALAXY,0.932346,9149,58039,775
4,1.237680e+18,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,8102,301,3,137,6.891865e+18,GALAXY,0.116123,6121,56187,842


In [4]:
data = data.drop(columns=["obj_ID","spec_obj_ID","run_ID", "rerun_ID", "cam_col", "field_ID", "plate", "MJD", "fiber_ID"])

In [5]:
data.head()

,alpha,delta,u,g,r,i,z,class,redshift
0,135.689107,32.494632,23.87882,22.27530,20.39501,19.16573,18.79371,GALAXY,0.634794
1,144.826101,31.274185,24.77759,22.83188,22.58444,21.16812,21.61427,GALAXY,0.779136
2,142.188790,35.582444,25.26307,22.66389,20.60976,19.34857,18.94827,GALAXY,0.644195
3,338.741038,-0.402828,22.13682,23.77656,21.61162,20.50454,19.25010,GALAXY,0.932346
4,345.282593,21.183866,19.43718,17.58028,16.49747,15.97711,15.54461,GALAXY,0.116123


In [6]:
data.dropna(inplace = True)

In [7]:
data.isnull().sum()

,0
alpha,0
delta,0
u,0
g,0
r,0
i,0
z,0
class,0
redshift,0


In [8]:
X = data.drop("class", axis=1)
y = LabelEncoder().fit_transform(data["class"])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [10]:
Xgb = XGBClassifier(random_state=42)

In [11]:
MODEL = Xgb.fit(X_train, y_train)

In [12]:
y_predict_base = MODEL.predict(X_test)

In [13]:
base_accuracy_score = accuracy_score(y_test, y_predict_base)

In [14]:
print(base_accuracy_score)

0.9763


In [50]:
scale = StandardScaler()
X_train_s = scale.fit_transform(X_train)
X_test_s = scale.transform(X_test)

In [51]:
MODEL_s = Xgb.fit(X_train_s, y_train)

In [53]:
y_predict_base_s = MODEL_s.predict(X_test_s)

In [54]:
base_accuracy_score_s = accuracy_score(y_test, y_predict_base_s)

In [55]:
print(base_accuracy_score_s)

0.97625


In [33]:
param_grid = {"n_estimators": [200],
              "learning_rate": [0.1],
              "max_depth": [5],
              "subsample": [0.8],
              "colsample_bytree": [1.0],
              "random_state": [42]
              }

In [36]:
xgb_clf_best = XGBClassifier(random_state=42)

In [35]:
grid = GridSearchCV(xgb_clf_best, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1.0], 'learning_rate': [0.1],
                         'max_depth': [5], 'n_estimators': [200],
                         'random_state': [42], 'subsample': [0.8]},
             scoring='accuracy')

In [37]:
y_pred_best = grid.predict(X_test)

In [40]:
print("Tuned Accuracy:", accuracy_score(y_test, y_pred_best))

Tuned Accuracy: 0.97575


In [41]:
print("Best Params:", grid.best_params_)

Best Params: {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'random_state': 42, 'subsample': 0.8}


In [46]:
grid = GridSearchCV(xgb_clf_best, param_grid, cv=3, scoring="accuracy", n_jobs=-1)
grid.fit(X_train_s, y_train)

GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, feature_weights=None,
                                     gamma=None, grow_policy=None,
                                     importance_type=None,
                                     interaction_constraints=Non...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [1.0], 'learning_rate': [0.1],
                         'max_depth': [5], 'n_estimators': [200],
                         'random_state': [42], 'subsample': [0.8]},
             scoring='accuracy')

In [48]:
y_pred_best_s = grid.predict(X_test_s)

In [49]:
print("Tuned Accuracy:", accuracy_score(y_test, y_pred_best_s))

Tuned Accuracy: 0.9761


In [56]:
print(grid.best_params_)

{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200, 'random_state': 42, 'subsample': 0.8}
